In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\Monu kumar\OneDrive\Desktop\End-to-end-Medical-Chatbot-using-Llama2\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from pinecone import Pinecone

# PINECONE_API_KEY = "8d8905b7-a0c9-4765-b687-639f837aa7dd"
# # PINECONE_API_ENV = "gcp-starter"
pc = Pinecone(api_key="8d8905b7-a0c9-4765-b687-639f837aa7dd")
index = pc.Index("medical-bot")

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("Data/")

In [7]:
# extracted_data

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
# make text chunks
text_chunks = text_split(extracted_data)
#text chunks iterable and metadatalist
text = [t.page_content for t in text_chunks]
metadata_list = [{"text": chunk.page_content} for chunk in text_chunks]

# print("length of my chunk:", len(text_chunks))

In [10]:
# text_chunks


In [11]:

#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:

model = download_hugging_face_embeddings()

In [13]:
model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query_result = model.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [16]:
text_chunks[5]

Document(page_content='mation presented in this publication, the Gale Group neither guaranteesthe accuracy of the data contained herein nor assumes any responsibili-ty for errors, omissions or discrepancies. The Gale Group accepts nopayment for listing, and inclusion in the publication of any organiza-tion, agency, institution, publication, service, or individual does notimply endorsement of the editor or publisher. Errors brought to theattention of the publisher and verified to the satisfaction of the publish-er', metadata={'source': 'Data\\Medical_book.pdf', 'page': 3})

In [17]:
embeddings_list = [model.embed_query(text) for text in text]

In [18]:
# from pinecone import Pinecone done

# Initialize Pinecone instance
# pc = Pinecone(api_key="8d8905b7-a0c9-4765-b687-639f837aa7dd")
# index = pc.Index("medical-bot") done

# Extract text content from text chunks
# text_content_list = [t.page_content for t in text_chunks]
# metadata_list = [{"text": chunk.page_content} for chunk in text_chunks]done

# Generate embeddings for the text chunks
# # model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2") done
# index_name="medical-bot"

# # Create vectors
# vectors = []
# for i, emb in enumerate(embeddings_list):
#     vector = {
#         "id": f"vec{i}",
#         "values": emb , # Convert embedding tuple to list
#         "metadata": metadata_list[i]
#     }
#     vectors.append(vector)
# batch_size=100
# # Upsert vectors in batches
# for i in range(0, len(vectors), batch_size):
#     batch = vectors[i:i+batch_size]
#     index.upsert(batch, index_name=index_name, namespace="ns1")
# index.upsert(vectors, index_name=index_name, namespace="ns1")

In [19]:
# #If we already have an index we can load it like this
# docsearch=Pinecone.from_existing_index(index_name, embeddings)
# from pinecone import Pinecone
query_result = model.embed_query("What are Allergies")

# docs=docsearch.similarity_search(query, k=3)

# print("Result", docs)
# pc = Pinecone(api_key="8d8905b7-a0c9-4765-b687-639f837aa7dd")
# index = pc.Index("medical-bot")

result = index.query(
    namespace="ns1",
    vector=query_result,
    top_k=3,  # Set top_k to 1 to retrieve only the top 1 match
    include_values=True,
    include_metadata=True,
)
result
# Extract text from the top match (if any)
if result['matches']:  # Check if there are any matches returned
    top_match = result['matches'][0]  # Access the first (top) match
    top_text = top_match['metadata']['text']
   
    print(top_text)
else:
    print("No matches found.")

GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE


In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
# chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm=CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Monu kumar\AppData\Local\Temp\ipykernel_16184\3597584327.py:1: SyntaxWarning: invalid escape sequence '\l'
  llm=CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",


In [24]:
# from langchain.vectorstores import Pinecone
# index = pc.Index("medical-bot")

# text_field = "text"
# vectorstore = Pinecone(
#     index, model.embed_query,text_field
# from langchain.vectorstores import Pinecone
# index = pc.Index("medical-bot")

# text_field = "text"
# vectorstore = Pinecone(
#     index, model.embed_query,text_field
# )
from langchain.vectorstores import Pinecone 
text_field = "text"
vectorstore = Pinecone(
    index=index,
    embedding_function=model.embed_query,
    text_key=text_field
)

In [25]:

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(top_k=2),
    return_source_documents=True, 
    chain_type_kwargs={"prompt": PROMPT}
)

In [26]:
# while True:
#     user_input = input("Input Prompt: ")
#     query_vector = model.embed_query(user_input)  # Embed the user input into a vector
#     result = index.query(
#         vector=query_vector,  # Provide vector as a keyword argument
#         top_k=2,  # Specify top_k as a keyword argument
#         namespace='ns1',  # Specify namespace as a keyword argument
#         include_values=True,
#         include_metadata=True,
#     )
# if result['matches']:  # Check if there are any matches returned
#     top_match = result['matches'][0]  # Access the first (top) match
#     top_text = top_match['metadata']['text']
   
#     print(top_text)
# else:
#     print("No matches found.")


In [27]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=index.query(vector=model.embed_query(user_input), top_k=2, namespace='ns1')
#     print(result)

In [31]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"input": user_input})
    print("Response : ", result["result"])

ValueError: `run` not supported when there is not exactly one output key. Got ['result', 'source_documents'].